### Evaluating different runs

In [38]:
%pylab --no-import-all
%matplotlib inline

import numpy as np
import pandas as pd
import os
import bcolz
import itertools
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.models import model_from_json
from datetime import datetime
import json

def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [39]:
rundir = "../model/runs/"

In [40]:
model_runs = os.listdir(rundir)
model_runs.sort()

model_runs = [ fi for fi in model_runs if  fi.endswith(".confusion") ]
model_names =   [os.path.splitext(x)[0] for x in model_runs] 

model_names

['basic2d.2017-01-27-16-14-37']

In [41]:
meta = []
for name in model_names:
    path = os.path.join(rundir,name +".meta")

    with open(path, 'r') as f:
        data = json.load(f)
        meta.append(data)
    

In [42]:
print(meta)

[{'2d run': '2d run', 'shape': '(64, 64, 64)'}]


In [43]:
mlayers = []
for name in model_names:
    path = os.path.join(rundir,name + ".model")
    print(path)

    with open(path, 'r') as f:
        #data = f.read()
        data = json.load(f)

        layers = []
        confing = data["config"]
        for x in confing:
            #print(x)
            base = x["class_name"]
            if "config" in x:
                config = x["config"]
                for key in ["nb_filter","nb_row","nb_col"]:
                    if key in config:
                        base = base +", " + key +":" + str(config[key])

            #print()
            layers.append(base)
        mlayers.append("|".join(layers))
    #model = model_from_json(data)
    #model.summary()


../model/runs/basic2d.2017-01-27-16-14-37.model


In [44]:
tp = []; fp = [] ; fn = []; tn = []; accuracy = []

for name in model_names:
    path = os.path.join(rundir,name + ".confusion")
    cm = load_array(path)
    tp.append(cm[0,0])
    fp.append(cm[0,1])
    fn.append(cm[1,0])
    tn.append(cm[1,1])
    accuracy.append((cm[0,0] + cm[1,1] )/ float(cm[0,0] + cm[1,1] + cm[1,0] + cm[0,1] ))
    
tp

[343]

In [45]:
dataset = pd.DataFrame.from_dict({"name":model_names,"meta":meta,"tp":tp,"fp":fp,"fn":fn,"tn":tn,"accuracy":accuracy,'layers':mlayers})

In [46]:
dataset.head()


,accuracy,fn,fp,layers,meta,name,tn,tp
0,0.742424,119,0,"Lambda|ZeroPadding2D|Convolution2D, nb_filter:...","{'2d run': '2d run', 'shape': '(64, 64, 64)'}",basic2d.2017-01-27-16-14-37,0,343


In [47]:
dataset.to_csv(os.path.join(rundir,"totals.csv"))